In [ ]:
# Environment & determinism (ensemble demo)
import os, sys, json, random
try:
    import numpy as np
except Exception:
    np = None
SEED = 123
random.seed(SEED)
if np is not None:
    np.random.seed(SEED)
print(json.dumps({'seed': SEED, 'ci': os.getenv('CI','0')=='1'}, indent=2))

# Restoria Ensemble Demo

Demonstrates combining multiple backends in Restoria for improved restoration quality.

## Optional extras (Colab only)
Install optional ORT CPU and metrics libraries if you're on Colab and want faster runs and additional metrics. Safe to skip.

In [ ]:
import os, sys, subprocess
IN_COLAB = "COLAB_GPU" in os.environ or "google.colab" in sys.modules
if IN_COLAB and not os.environ.get('NB_CI_SMOKE'):
    def pip_install(pkgs):
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)
        except Exception as e:
            print('[WARN] pip failed:', e)
    extras = [
        'onnxruntime>=1.17,<2',
        'lpips>=0.1.4',
        'git+https://github.com/richzhang/PerceptualSimilarity.git@master#egg=lpips-extra',
        "torchdists>=0.1.0; python_version>='3.9'",
    ]
    print('[INFO] Installing optional extras…')
    pip_install(extras)
else:
    print('[INFO] Extras install skipped (not in Colab or CI).')

# Ensemble Backend Demo (Dry-Run)

This notebook shows how to invoke the GFPP CLI programmatically to simulate an ensemble run without downloading models.

Notes:
- We use a small sample image from `assets/` and enable `--dry-run` so no weights are downloaded.
- You can tweak `--ensemble-backends` and `--ensemble-weights` safely in dry-run mode to see the planned config.
- To actually restore faces, remove `--dry-run` and install the appropriate extras per the docs.

In [ ]:
# Minimal imports (avoid heavy libraries in dry-run)
from src.gfpp.cli import cmd_run
import os

# Inputs/outputs
inp = os.path.join('assets','gfpgan_logo.png')
out = os.path.join('results','dry_demo','notebooks','ensemble')
os.makedirs(out, exist_ok=True)

# Ensemble configuration (safe in dry-run)
args = [
    '--input', inp,
    '--output', out,
    '--backend', 'ensemble',
    '--ensemble-backends', 'gfpgan,codeformer,restoreformerpp',
    '--ensemble-weights', '0.6,0.2,0.2',
    '--metrics', 'off',
    '--dry-run'
]

code = cmd_run(args)
print('exit', code)
print('Planned results directory:', out)

### Optional: compute fast metrics
Run a short fast-metrics pass (if dependencies are available). Skipped automatically in CI and when metrics packages are missing.

In [ ]:
import os, sys
if not os.environ.get('NB_CI_SMOKE'):
    try:
        if 'restoria' not in sys.modules:
            import sys as _sys
            _sys.path.insert(0, '/content/Restoria/src') if '/content' in _sys.path[0] else None
        from restoria.cli.main import main as restoria_main
        img = globals().get('img', 'assets/gfpgan_logo.png')
        out_dir = globals().get('out_dir', 'results/dry_demo/ensemble_demo')
        restoria_main(['run','--input', img,'--output', out_dir,'--device','auto','--metrics','fast'])
    except Exception as e:
        print('[WARN] fast metrics run skipped:', e)
else:
    print('[INFO] Skipped in CI smoke mode.')

In [ ]:
# Optional: inspect manifest if written during dry-run
from glob import glob
import json

manifests = glob(os.path.join(out, 'manifest*.json'))
print('Found manifests:', manifests)
if manifests:
    try:
        with open(manifests[0], 'r') as f:
            data = json.load(f)
        print('env.runtime keys:', list((data.get('env', {}) or {}).get('runtime', {}).keys()))
        print('images recorded:', len(data.get('images') or []))
    except Exception as e:
        print('Manifest read skipped:', e)

In [ ]:
# Quickstart (dry-run) ensemble demo
import os, sys
from pathlib import Path
if 'restoria' not in sys.modules:
    sys.path.insert(0, str(Path.cwd() / 'src'))
from restoria.cli.main import main as restoria_main
img = 'assets/gfpgan_logo.png' if os.path.exists('assets/gfpgan_logo.png') else 'tmp/blank.png'
out_dir = 'results/dry_demo/ensemble'
rc = restoria_main(['run','--input', img,'--output', out_dir,'--dry-run','--device','auto'])
print('rc:', rc)

In [ ]:
# Optional: inspect manifest if written during dry-run
import json, os
mp = os.path.join('results/dry_demo/ensemble','manifest.json')
if os.path.exists(mp):
    m = json.load(open(mp))
    print('device:', m.get('device'))

### Real run (optional)

Try a real run and compare outputs across backends by repeating with different `--backend` values:

```python
from restoria.cli.main import main as restoria_main
img = 'assets/gfpgan_logo.png'
out_dir = 'results/real_demo/ensemble'
for be in ['gfpgan','gfpgan-ort','codeformer']:
    restoria_main(['run','--input', img,'--output', f"{out_dir}_{be}",'--device','auto'])
```

## Plan and outputs summary
Use this to quickly view device, chosen backend, and a sample of metrics after your runs.

In [ ]:
import json, os

def summarize_out(out_dir):
    man = os.path.join(out_dir, 'manifest.json')
    met = os.path.join(out_dir, 'metrics.json')
    if os.path.exists(man):
        m = json.load(open(man))
        env = m.get('env', {})
        plan = m.get('plan', {})
        print('device:', env.get('resolved_device') or env.get('device'))
        if plan:
            print('backend:', plan.get('backend'), '| reason:', plan.get('reason'))
    else:
        print('manifest.json missing')
    if os.path.exists(met):
        data = json.load(open(met))
        rows = data.get('records') or []
        print('metrics records:', len(rows))
        if rows:
            m0 = rows[0].get('metrics') or {}
            for k in ['arcface_cosine','lpips','dists']:
                if k in m0:
                    print(k, '=', m0[k])
    else:
        print('metrics.json missing')

out_dir = globals().get('out_dir', 'results/dry_demo/ensemble_demo')
try:
    summarize_out(out_dir)
except Exception as e:
    print('[WARN] summary failed:', e)